# A notebook to read a CSV template with extract info and load it to elab

Please enter file name in the following cell, and what is the delimiter in that file

In [1]:
CSVfilename="/Users/pierrespc/Documents/PostDocPasteur/aDNA/Import_eLAB/API_FUNCTIONALITIES/UploadData/extract/RawTables/uploadExtracts_fromCSV_20211201.txt"
delimiterFile="\t"

Please enter the one-line file where your token is saved in the following cell

In [2]:
tokenFile="/Users/pierrespc/Documents/PostDocPasteur/aDNA/Import_eLAB/API_FUNCTIONALITIES/credentials/tokenELAB"

Now preparing all required python libs

In [3]:
import os
import json
import requests
import csv
import pandas
import numpy
from apiclient import discovery, errors
from httplib2 import Http
from oauth2client import client, file, tools
import os.path

token = format(open(tokenFile,"r").readline().strip())
url = "https://elab-dev.pasteur.fr/api/v1/"
headers1 = {'Authorization': token, 'Accept': 'application/json','Content-Type':'application/json'}
headers2 = {'Authorization': token, 'Accept': 'application/json'}
params={}




Reading the data. ExeDict is to make sure thta if eLab configuration change we justy need to change here and not the excel template. 

In [4]:
ExeDict={"Name":"ExtractID",
         "From Skeleton Element":"RascovanLabID",
         "Description":"Description",
         "Note":"Notes",
         "Amount":"Weight",
         "Unit":"fixed_gram",
         "parentSampleID":"RascovanLabID",
         "Date of drilling":"Date",
         "Pictures":"Pictures",
         "Person in charge":"PersonInCharge",
         "Laboratory where processed":"Laboratory",
         "Extract Type":"ExtractType",
         "Conservation":"Observation",
         "Pathology":"Pathologie",
         "Pathology description":"None",
         "Taken for extraction":"TakenForExtraction",
         "Extracted":"Extraction",
         "Extraction Comment":"extractionComment",
         "density UDG treatment (ng/uL)":"densityUDGtreated",
         "Volume UDG treatment (uL)":"volumeUDGtreated",
         "mass UDG in Tube (ng)":"massInTube"
}

extractTable=pandas.read_csv(CSVfilename,delimiter=delimiterFile)



Prepare all the eLab-API keys necessary to down and upload data

In [5]:
def BadRequest(myReq,code=200):
    return(myReq.status_code !=code)


r = requests.get(url + "sampleTypes", headers = headers2)
if BadRequest(r,200):
    print("Bad request")
data = r.json()
types = {}
for typ in data.get("data"):
    types[format(typ.get("name"))] = format(typ.get("sampleTypeID"))

print(types)

r = requests.get(url + "sampleTypes/" + types["Extract"] + "/meta", headers = headers2)
if BadRequest(r,200):
    print("Bad request")
data = r.json()
FeateLabExe = {}
for feat in ['Name','Description','Note','Amount','Unit',"parentSampleID"]:
    FeateLabExe[feat] = {"ID": "notMeta"}
for feat in data.get("data"):
    FeateLabExe[format(feat.get("key"))] = { "ID":format(feat.get("sampleTypeMetaID")),
                                              "TYPE":format(feat.get("sampleDataType"))}

{'Individual': '39466', 'Site': '39468', 'Skeleton Element': '39469', 'Extract': '39470', 'Indexed Library': '39494', 'Library pool': '39495', 'Non Indexed Library': '39556', 'Bone pellet': '39599'}


Fill RascovanLabID or extractID columns in the table

In [6]:
###get number of extracts  for skeleton elements
r = requests.get(url + "samples" , headers = headers2, params = {'sampleTypeID': types["Extract"]})
BadRequest(r,200)
data = r.json()
extractEntered = {}
for sam in data.get("data"):
    extract=format(sam.get("name"))
    rasID=".".join(extract.split(".")[0:2])
    if rasID not in extractEntered.keys():
        extractEntered[rasID]=1
    else:
        extractEntered[rasID]=extractEntered[rasID]+1

#make "From Skeleton Element column" or extract column
###change date format if needed
for index,name in extractTable["ExtractID"].items():
    ###case need to make RascovanLabID
    if format(name) != "nan" and format(extractTable.loc[index,"RascovanLabID"]) == "nan":
        extractTable.loc[index,"RascovanLabID"]=".".join(name.split(".")[0:2])
    ###case need to make extract
    elif format(name) == "nan" and format(extractTable.loc[index,"RascovanLabID"]) != "nan":
        if extractTable.loc[index,"RascovanLabID"] not in extractEntered.keys():
            numExt=1
        else:
            numExt=extractEntered[extractTable.loc[index,"RascovanLabID"]]+1
        extractEntered[extractTable.loc[index,"RascovanLabID"]]=numExt
        if numExt < 10:
            numExt="0"+str(numExt);
        else:
            numExt=str(numExt)
        extractTable.loc[index,"ExtractID"]=extractTable.loc[index,"RascovanLabID"]+"."+numExt
    elif format(name) == "nan" and format(extractTable.loc[index,"RascovanLabID"]) == "nan":
        print("must enter an extract or a skeleton id")
        print(extractTable.loc[index,])
        raise()
    else:
        print("can not enter an extract AND a skeleton id")
        print(extractTable.loc[index,])
        raise()
        
    
    if "/" in format(extractTable.loc[index,"Date"]):
        extractTable.loc[index,"Date"]="-".join(extractTable.loc[index,"Date"].split("/")[::-1])
print("finished")
extractTable

finished


,RascovanLabID,ExtractID,Date,ExtractType,Weight,TakenForExtraction,Freezer,Observation,Pathologie,GeneralSampleComment,...,densityUDGtreated,volumeUDGtreated,massInTube,Pictures,Notes,Description,DrillingProtocole,ExtractionProtocole,PersonInCharge,Laboratory
0,AR0079.1,AR0079.1.01,2021-11-30,dental calculus,0.001,yes,"Hannes's freezer, bag mix batch, sub-bag Maria...",none,no,"Mariano del Papa, collected from zip-lock bags",...,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN
1,AR0080.1,AR0080.1.01,2021-11-30,dental calculus,0.003,yes,"Hannes's freezer, bag mix batch, sub-bag Maria...",none,no,"Mariano del Papa, collected from zip-lock bags",...,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN
2,AR0081.1,AR0081.1.01,2021-11-30,dental calculus,0.011,yes,"Hannes's freezer, bag mix batch, sub-bag Maria...",none,no,"Mariano del Papa, collected from zip-lock bags",...,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN
3,AR0082.1,AR0082.1.01,2021-11-30,dental calculus,<0.001,yes,"Hannes's freezer, bag mix batch, sub-bag Maria...",does not weight much but it's there,no,"Mariano del Papa, collected from zip-lock bags",...,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN
4,AR0083.1,AR0083.1.01,2021-11-30,dental calculus,0.006,yes,"Hannes's freezer, bag mix batch, sub-bag Maria...",none,no,"Mariano del Papa, collected from zip-lock bags",...,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN
5,AR0084.1,AR0084.1.01,2021-11-30,dental calculus,0.003,yes,"Hannes's freezer, bag mix batch, sub-bag Maria...",none,no,"Mariano del Papa, collected from zip-lock bags",...,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN
6,AR0085.1,AR0085.1.01,2021-11-30,dental calculus,0.011,yes,"Hannes's freezer, bag mix batch, sub-bag Maria...",none,no,"Mariano del Papa, collected from zip-lock bags",...,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN
7,AR0086.1,AR0086.1.01,2021-11-30,dental calculus,0.001,yes,"Hannes's freezer, bag mix batch, sub-bag Maria...",none,no,"Mariano del Papa, collected from zip-lock bags",...,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN
8,AR0087.1,AR0087.1.01,2021-11-30,dental calculus,0.016,yes,"Hannes's freezer, bag mix batch, sub-bag Maria...",none,no,"Mariano del Papa, collected from zip-lock bags",...,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN
9,AR0088.1,AR0088.1.01,2021-11-30,dental calculus,0.013,yes,"Hannes's freezer, bag mix batch, sub-bag Maria...",none,no,"Mariano del Papa, collected from zip-lock bags",...,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN


Assign some columns that are fixed (e.g. drilling protocoles, person in charge).

In [7]:
r = requests.get(url + "sampleTypes/" + types["Extract"] + "/meta/", headers = headers2)
if BadRequest(r,200):
    r.raise_for_status()

r=r.json().get("data")

for FixedColumn in ["Person in charge","Laboratory where processed"]:
    listPossible=""
    if len(extractTable[ ExeDict[FixedColumn]].unique()) == 1:
        for feat in r:
            if feat["key"] ==  FixedColumn:
                if feat["sampleDataType"]=="COMBO" or feat["sampleDataType"]=="CHECKBOX":
                    listPossible=feat["optionValues"]
                else:
                    listPossible="Anything"
                break
        prompt = input("put an unique value for "+FixedColumn+"\npossibilities are "+" / ".join(listPossible))
        while prompt not in listPossible and listPossible != "Anything" :
            prompt = input("put an unique value for "+FixedColumn+"\npossibilities are "+" / ".join(listPossible))
        extractTable[ExeDict[FixedColumn]]=prompt
  



put an unique value for Person in charge
possibilities are Miren Iraeta Orbegozo / Maria Lopopolo / NA Maria Lopopolo
put an unique value for Laboratory where processed
possibilities are Hannes Schroeder Hannes Schroeder


In [8]:
extractTable

,RascovanLabID,ExtractID,Date,ExtractType,Weight,TakenForExtraction,Freezer,Observation,Pathologie,GeneralSampleComment,...,densityUDGtreated,volumeUDGtreated,massInTube,Pictures,Notes,Description,DrillingProtocole,ExtractionProtocole,PersonInCharge,Laboratory
0,AR0079.1,AR0079.1.01,2021-11-30,dental calculus,0.001,yes,"Hannes's freezer, bag mix batch, sub-bag Maria...",none,no,"Mariano del Papa, collected from zip-lock bags",...,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN,Maria Lopopolo,Hannes Schroeder
1,AR0080.1,AR0080.1.01,2021-11-30,dental calculus,0.003,yes,"Hannes's freezer, bag mix batch, sub-bag Maria...",none,no,"Mariano del Papa, collected from zip-lock bags",...,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN,Maria Lopopolo,Hannes Schroeder
2,AR0081.1,AR0081.1.01,2021-11-30,dental calculus,0.011,yes,"Hannes's freezer, bag mix batch, sub-bag Maria...",none,no,"Mariano del Papa, collected from zip-lock bags",...,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN,Maria Lopopolo,Hannes Schroeder
3,AR0082.1,AR0082.1.01,2021-11-30,dental calculus,<0.001,yes,"Hannes's freezer, bag mix batch, sub-bag Maria...",does not weight much but it's there,no,"Mariano del Papa, collected from zip-lock bags",...,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN,Maria Lopopolo,Hannes Schroeder
4,AR0083.1,AR0083.1.01,2021-11-30,dental calculus,0.006,yes,"Hannes's freezer, bag mix batch, sub-bag Maria...",none,no,"Mariano del Papa, collected from zip-lock bags",...,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN,Maria Lopopolo,Hannes Schroeder
5,AR0084.1,AR0084.1.01,2021-11-30,dental calculus,0.003,yes,"Hannes's freezer, bag mix batch, sub-bag Maria...",none,no,"Mariano del Papa, collected from zip-lock bags",...,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN,Maria Lopopolo,Hannes Schroeder
6,AR0085.1,AR0085.1.01,2021-11-30,dental calculus,0.011,yes,"Hannes's freezer, bag mix batch, sub-bag Maria...",none,no,"Mariano del Papa, collected from zip-lock bags",...,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN,Maria Lopopolo,Hannes Schroeder
7,AR0086.1,AR0086.1.01,2021-11-30,dental calculus,0.001,yes,"Hannes's freezer, bag mix batch, sub-bag Maria...",none,no,"Mariano del Papa, collected from zip-lock bags",...,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN,Maria Lopopolo,Hannes Schroeder
8,AR0087.1,AR0087.1.01,2021-11-30,dental calculus,0.016,yes,"Hannes's freezer, bag mix batch, sub-bag Maria...",none,no,"Mariano del Papa, collected from zip-lock bags",...,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN,Maria Lopopolo,Hannes Schroeder
9,AR0088.1,AR0088.1.01,2021-11-30,dental calculus,0.013,yes,"Hannes's freezer, bag mix batch, sub-bag Maria...",none,no,"Mariano del Papa, collected from zip-lock bags",...,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN,Maria Lopopolo,Hannes Schroeder


Run the next block  if you want to enter an unique Drilling Protocole

In [9]:
extractTable["DrillingProtocole"]=input("enter an unique Drilling Protocole for that extracts?")

enter an unique Drilling Protocole for that extracts? Del Papa


Check the columns in extract Table are recognized here. If no lines in output, you are just fine.

In [10]:
for feat in FeateLabExe.keys():
    if feat not in ExeDict.keys():
        print(feat + "--> NOT IN DICTIONARY")
        
for feat in ExeDict.keys():
    if feat not in FeateLabExe.keys():
        print(feat + "--> NOT IN eLAB")

We get all the possible values for checkboxes and dropdown features of Extracts and check our extractTable table is fine. If no lines in output, you're just fine

In [11]:
r = requests.get(url + "sampleTypes/" + types["Extract"] + "/meta", headers = headers2)
data = r.json()
for feat in data.get("data"):
    if feat.get("sampleDataType") == "CHECKBOX" or feat.get("sampleDataType") == "COMBO":
        OptionELAB=feat.get("optionValues")
        key=feat.get("key")
        if ExeDict[key].startswith("fixed"):
            tabVal=ExeDict[key].split("_")[1]
            if tabVal not in OptionELAB:
                print("--" + tabVal + "-- not mapped in eLab for " + key)
        else:
            extractTable.loc[extractTable[ExeDict[key]].isnull(),ExeDict[key]]="NA"
            for tabVal in extractTable[ExeDict[key]].unique():
                if tabVal not in OptionELAB:
                    print("--" + tabVal + "-- not mapped in eLab for " + key)



Now, we make the json for each extract and we upload or update in eLab!
Change the default prompt line:
- put y if you are sure you want to overwrite already loaded info in eLab, 
- put n if you are sure you want to leave already loaded info in eLab (although it doesn't match info in your table)
- put anything else if you want a case by case prompt

In [15]:
defaultPrompt="?"

####get all registered skeleton element and extracts
registered = {}
for name in ["Skeleton Element","Extract"]:
    print(name)
    r = requests.get(url + "samples" , headers = headers2, params = {'sampleTypeID': types[name]})
    BadRequest(r,200)
    data = r.json()
    myList = {}
    for sam in data.get("data"):
        if format(sam.get("name")) in myList.keys():
            print(name + ": " + sam.get("name") + " duplicated")
            break
        myList[format(sam.get("name"))]=format(sam.get("sampleID"))
    registered[name] = myList



###iterate over extracts in table
for index,name in extractTable[ExeDict['Name']].items():
    if format(name)=="nan":
        continue
    del(r)
    patch=False
        
    ###get what is already uploaded fior that extract in eLab
    if name in registered['Extract'].keys():
        patch=True
        id=registered['Extract'][name]
        r=requests.get(url + "samples/"+id, headers = headers2)
        BadRequest(r,200)
        r=r.json()
        
        ###change to lower case all the keys because API sometimes use upper, lower for different request (A MESS!)
        dataLoaded={}
        for oldkey in r:
            newkey=oldkey.lower()
            dataLoaded[newkey] = r[oldkey]
    ####prepare the Data to be loaded
    Data={}
    for fea in FeateLabExe.keys():
        if FeateLabExe[fea]['ID'] == "notMeta" and fea not in ["Amount","Unit"]:
            ###fixed value (from dico)
            if ExeDict[fea].startswith("fixed"):
                element=ExeDict[fea].split("_")[1]
            elif ExeDict[fea]=="None":
                element="Nothing entered"
            elif fea == "parentSampleID":
                if not name.startswith("Blank"):
                    element=registered["Skeleton Element"][extractTable["RascovanLabID"][index]]
                else:
                    element=None
            else:
                element=extractTable[ExeDict[fea]][index]

            ###check delta when patching
            if patch:
                elementLoaded=dataLoaded[fea.lower()]
                if format(elementLoaded) != format(element):
                    print("For extract "+name+", do you want to update the "+fea+" field? That is: "+format(element)+ " vs what already loaded: "+format(elementLoaded))
                    prompt=defaultPrompt
                    while prompt not in ["y","n"]:
                        prompt = input("replace y/n??")
                    print(prompt)
                    if prompt == "n":
                        element=elementLoaded
            Data[fea]=element
    ###case of updating
    if patch:
        DR=requests.patch(url + "samples/"+id, headers = headers2,data = Data)
        if BadRequest(DR,204):
            DR.raise_for_status()
    else:
        ###case of uploading
        #print(name + "uploading")
        patch=False
        Data["sampleTypeID"]=types["Extract"]
        Data["Name"]=name
        DR=requests.post(url + "samples/", headers = headers2,data = Data)   
    ####check the Data loading was correct
    if BadRequest(DR,204):
        DR.raise_for_status()
    ###actualize the registered["Site"] list (checking we did not duplicated anything here)
    r=requests.get(url + "samples/forNames?names="+name, headers = headers2)
    if BadRequest(r,200):
        r.raise_for_status()
    data=r.json()
    sam=data.get("data")
    if len(sam)!=1:
        print("different Extract entries (" + str(len(sam)) + ") for name "+name)
        break
    else:
        sam=sam[0]
        id=str(sam.get("sampleID"))
        #print("Data OK for "+ name + " (" + id + ")")
        registered["Extract"][name]=id

    ###patch the metaData
    if patch:
        #print("patching meta so need to heck if differences for "+name)
        MDR=requests.get(url + "samples/"+id+"/meta", headers = headers2)
        BadRequest(MDR,200)
        data=MDR.json().get("data")
        metaLoaded={}
        for i in data:
            metaLoaded[i["key"]]=str(i["value"])

    for fea in FeateLabExe.keys():
        needToPatch=False
        MDR=requests.get(url + "samples/"+id+"/meta", headers = headers2)
        BadRequest(MDR,200)
        ###get new element to be loaded
        if FeateLabExe[fea]['ID'] != "notMeta" and FeateLabExe[fea]['TYPE'] != "FILE":
            ###fixed value (from dico)
            if ExeDict[fea].startswith("fixed"):
                element=ExeDict[fea].split("_")[1]
                MetaData={"key": fea,
                          "sampleTypeMetaID": int(FeateLabExe[fea]['ID']),
                          "value": element,
                          "sampleDataType": FeateLabExe[fea]['TYPE']}
            elif ExeDict[fea]=="None":
                element="Nothing entered"
                MetaData={"key": fea,
                          "sampleTypeMetaID": int(FeateLabExe[fea]['ID']),
                          "value": element,
                          "sampleDataType": FeateLabExe[fea]['TYPE']}
            elif fea == "From Skeleton Element":
                if not name.startswith("Blank"):
                    sisi=extractTable["RascovanLabID"][index]
                    IDsisi=registered["Skeleton Element"][sisi]
                    element=sisi+"|"+IDsisi
                    samples={"sampleID": IDsisi,"name": sisi}
                else:
                    samples=[]
                    splitted=extractTable["extractionComment"][index].split(",")
                    splitted=list(dict.fromkeys(splitted))
                    for sisi in splitted:
                        IDsisi=registered["Extract"][sisi]
                        samples.append({"sampleID": IDsisi,"name": sisi})
                        if sisi != splitted[0]:
                            element=element+"|"+sisi+"|"+IDsisi
                        else:
                            element=sisi+"|"+IDsisi
                MetaData={
                    "sampleTypeMetaID": int(FeateLabExe[fea]['ID']),
                    "sampleDataType": FeateLabExe[fea]['TYPE'],
                    "samples": samples,
                    "key": fea,
                    "value": element
                }
            elif fea == "Pictures":
                pictures=extractTable[ExeDict[fea]][index]
                if pictures == "T":
                    element="/pasteur/entites/metapaleo/Research/ERC-project/Samples/pictures/Drilling/"+extractTable["RascovanLabID"][index]
                else:
                    element="None"
                MetaData={"key": fea,
                          "sampleTypeMetaID": int(FeateLabExe[fea]['ID']),
                          "value": element,
                          "sampleDataType": FeateLabExe[fea]['TYPE']}
            else:
                element=extractTable[ExeDict[fea]][index]
                if format(element)=="nan" or format(element)=="" or format(element)==" ":
                    element="Nothing entered"
                MetaData={"key": fea,
                          "sampleTypeMetaID": int(FeateLabExe[fea]['ID']),
                          "value": element,
                          "sampleDataType": FeateLabExe[fea]['TYPE']}
            
            ###check if this is a new entry or not
            if patch:
                ###check if new element is similar to what already loaded
                if fea not in metaLoaded.keys(): 
                    needToPatch=True
                elif metaLoaded[fea] != str(element):
                    print("difference for " + name + "(feature: " + fea + ") " + element + " vs loaded : " + metaLoaded[fea])
                    prompt=defaultPrompt
                    while prompt not in ["y","n"]:
                        prompt = input("???replace y/n??")
                    if prompt == "y":
                        needToPatch=True
            else:
                needToPatch=True
                        
            if needToPatch:
                #print(MetaData)      
                MDR=requests.put(url + "samples/"+id+"/meta", headers = headers2,data = MetaData)
                ####check the MetaData loading was correct
                BadRequest(MDR,204)
    #print("metadata OK for "+ name + " (" + id + ")")
    ###patch the quantity
    Quant={}
    Note=None
    for fea in ['Amount','Unit']:
        if ExeDict[fea].startswith("fixed"):
            element=ExeDict[fea].split("_")[1]
        elif ExeDict[fea]=="None":
            element="Nothing entered"
        else:
            element=extractTable[ExeDict[fea]][index]
            if format(element)=="nan":
                element=0
            elif "<" in format(element):
                Note="actual weight reported: "+element
                element=0
        Quant[fea]=element
    Quant["displayUnit"]=Quant["Unit"].capitalize()
    Quant["fullAmount"]=Quant["Amount"]
    QR=requests.put(url + "samples/" + id + "/quantity", headers = headers2, data = Quant)
    BadRequest(QR,204)

    ###put actual weight in note when there is a "<"
    if Note is not None:
            r=requests.get(url + "samples/"+id, headers = headers2)
            BadRequest(r,200)
            Data=r.json()
            Data["note"]=Data["note"]+" / "+ Note
            r=requests.patch(url + "samples/"+id, headers = headers2,data = Data)
            BadRequest(r,204)
print("finished") 



Skeleton Element
Extract
For extract AR0082.1.01, do you want to update the Note field? That is: nan vs what already loaded: nan / actual weight reported: <0.001


replace y/n?? y


y
finished


## Extract assignation to experiments
First retrieve the eLab ID needed to access the sampleIN and sampleOUT sections.

In [16]:
r = requests.get(url + "experiments", headers = headers2,params = params)
data = r.json()
experiments = {}
for exp in data.get("data"):
    experiments[format(exp.get("name"))] = format(exp.get("experimentID"))



for expe in list(experiments.keys()):
    #print(expe)
    idExpe=experiments[expe]
    r=requests.get("https://elab-dev.pasteur.fr/api/v1/experiments/"+idExpe+"/sections",headers=headers1)
    if r.status_code != 200:
        print(r.status_code)
        print(r.raise_for_status())
    if r.json().get("recordCount") == 0:
        print("no record")
        continue
    SampleIN={}
    SampleOUT={}
    for data in r.json().get("data"):
        if data["sectionType"] == "SAMPLESIN":
            SampleIN[data["sectionHeader"]]=data["expJournalID"]
        elif data["sectionType"] == "SAMPLESOUT":
            SampleOUT[data["sectionHeader"]]=data["expJournalID"]
    experiments[expe]={"ID":idExpe,
                      "sampleIN":SampleIN,
                      "sampleOUT":SampleOUT}
#print(experiments)

### Assign to Drilling /LAB/ Laboratory Protocols,
Where /LAB/ is the Lab appearing in DrillingProtocole colum
"pulverized pieces" (sampleOUT) and  the skeleton elements they derive from (sampleIN)

We start retrieving all field IDs required for that

In [22]:
#the sampleIN and sampleOUT id for the experiment
CorresExtract={"petrous":"Pulverized petrous bone",
                  "dental calculus":"Scratched Dental Calculus",
                  "pulp":"Pulverized Pulp",
                  "root":"Pulverized Root",
                   "root apex":"Pulverized Root Apex",
                  "long bone":"Pulverized long bone",
                    "other":"Pulverized other bone",
              }
CorresSkel={"Dental Calculus":"Tooth processed",
            "Petrous":"Petrous bone processed",
            "Tooth":"Tooth processed",
            "Other Bone":"Long bone processed "}

In [23]:
listOUT={}
listIN={}
for lab in ["Guraeib","Del Papa","Schroeder","Rascovan","Rascovan 2.0"]:
    listOUT[lab]={}
    listIN[lab]={}
    for exType in CorresExtract:
        listOUT[lab][CorresExtract[exType]]=[]

    for skelType in CorresSkel:
        listIN[lab][CorresSkel[skelType]]=[]


for index, extract in extractTable["ExtractID"].items():
    if format(extract)=="nan":
        continue
    protocole=extractTable["DrillingProtocole"][index]
    if protocole is None:
        continue
    idOUT=registered["Extract"][extract]
    MER=requests.get(url+"/samples/"+idOUT+"/meta",headers=headers1)
    BadRequest(MER,200)
    #get Extract Type and check it is found
    exType=None
    for meta in MER.json().get("data"):
        if meta["key"]=="Extract Type":
            exType=meta["value"]
            break
    if exType is None:
        print("Extract Type not found")
        break
    ###prepare sampleIN for that extract
    #get parentSampleID (the skeleton element)
    ER=requests.get(url+"/samples/"+idOUT,headers=headers1)
    if BadRequest(ER,200):
        ER.raise_for_status()
    idIN=format(ER.json()["parentSampleID"])

    #get meta
    SMR=requests.get(url+"/samples/"+idIN+"/meta",headers=headers1)
    if BadRequest(SMR,200):
        SMR.raise_for_status()

    ##get skeleton element type and check it is found
    archoID=None
    skelType=None
    expediente=None
    for meta in SMR.json().get("data"):
        if meta["key"]=="Bone type":
            skelType=meta["value"]
    if skelType is None:
        print("Skeleton Ele Type not found")
        print(SMR.json().get("data"))
        break

    listOUT[protocole][CorresExtract[exType]].append(idOUT)
    listIN[protocole][CorresSkel[skelType]].append(idIN)
    
#print(listIN)
#print(listOUT)
    

In [25]:
###upload sample IN
for lab in ["Guraeib","Del Papa","Schroeder","Rascovan","Rascovan 2.0"]:
    for type in listIN[lab].keys():
        data=listIN[lab][type]
        if len(data)==0:
            continue
            #print("sample IN : nothing to upload upload for "+type+" to "+lab)

        idIN=format(experiments["Drilling. "+lab+" Laboratory Protocols"]["sampleIN"][type])
        print("sample IN : upload for "+type+" to "+lab)
        data=format(data)
        r=requests.put(url+"/experiments/sections/"+idIN+"/samples",headers=headers1,data = data)
        BadRequest(r,204)

sample IN : upload for Tooth processed to Del Papa


In [26]:
###upload sample OUT
for lab in ["Guraeib","Del Papa","Schroeder","Rascovan","Rascovan 2.0"]:
    for type in listOUT[lab].keys():
        data=listOUT[lab][type]
        if len(data)==0:
            #print("sample OUT : nothing to upload upload for "+type+" to "+lab)
            continue        
        idOUT=format(experiments["Drilling. "+lab+" Laboratory Protocols"]["sampleOUT"][type])
        print("sample OUT : upload for "+type+" to "+lab)
        data=format(data)
        r=requests.put(url+"/experiments/sections/"+idOUT+"/samples",headers=headers1,data = data)
        if BadRequest(r,204):
            print("HU")


sample OUT : upload for Scratched Dental Calculus to Del Papa


### Assign to Extraction /LAB/ Laboratory Protocols,
Where /LAB/ is the Lab appearing in ExtractionProtocole colum
Now we add in as sampleIN and sampleOUT the "pulverized bone"


In [27]:
listIN={}
for lab in ["Schroeder","Rascovan","Rascovan 2.0"]:
    listIN[lab]=[]

for index, extract in extractTable["ExtractID"].items():
    if format(extract) == "nan":
        continue
    protocole=extractTable["ExtractionProtocole"][index]
    if format(protocole) == "nan":
        continue
    idIN=registered["Extract"][extract]
    listIN[protocole].append(idIN)
print(listIN)
for lab in ["Schroeder","Rascovan","Rascovan 2.0"]:
    data=format(listIN[lab])
    if format(data)=="[]":
        print("sample OUT : nothing to upload for extraction to "+lab)
    else:
        print(data)
        ###assign to sampleIN
        idExp={"c":str(value) for key, value in experiments["Extraction. "+lab+" Lab Protocols"]["sampleIN"].items()}["c"]
        r=requests.put(url+"/experiments/sections/"+idExp+"/samples",headers=headers1,data = data)
        if BadRequest(r,204):
            r.raise_for_status()
        ###assign to sampleOUT
            idExp={"c":str(value) for key, value in experiments["Extraction. "+lab+" Lab Protocols"]["sampleOUT"].items()}["c"]
        r=requests.put(url+"/experiments/sections/"+idExp+"/samples",headers=headers1,data = data)
        if BadRequest(r,204):
            r.raise_for_status()




print("finished")


{'Schroeder': [], 'Rascovan': [], 'Rascovan 2.0': []}
sample OUT : nothing to upload for extraction to Schroeder
sample OUT : nothing to upload for extraction to Rascovan
sample OUT : nothing to upload for extraction to Rascovan 2.0
finished


## Extract assignation to Storage
first organize the storage IDs (a bit messy but eLab treats all storage levels similarly for sample assignation)

In [35]:
storageByID={}
r=requests.get(url+"/storageLayers",headers=headers1)
if BadRequest(r,200):
    r.raise_for_status()
    
stoData=r.json().get("data")
for sto in stoData:
    
    storageByID[sto["storageLayerID"]]={"name":sto["name"],"parentID":sto["parentStorageLayerID"]}

def getParentSto(ID,stoDict):
    if stoDict[ID]["parentID"]==0:
        return(stoDict[ID]["name"])
    else:
        return(getParentSto(stoDict[ID]["parentID"],stoDict)+", "+stoDict[ID]["name"])
    
storage={}
storageRev={}
for stoID in storageByID.keys():
    name=getParentSto(stoID,storageByID)
    storage[name]=stoID
    storageRev[stoID]=name
    

Now assign extracts to that storage, accordingly to the "storageLayerID" column.

Change the default prompt line:
- put y if you are sure you want to move the extract in any case to what appears in the table
- put n if you are sure you want to leave the extract where it is in eLab (although it doesn't match info in your table)
- put anything else if you want a case by case prompt

In [36]:
defaultPrompt="?"
for index,name in extractTable[ExeDict['Name']].items():
    if format(name)=="nan":
        continue
    idEx=registered["Extract"][name]    
    freezer=extractTable["Freezer"][index]
    if format(freezer) in ["To be spotted","nan"] :
        freezer="Unknown"
    freezer=freezer.replace("Mariano Del Papa calculus to extract","Mariano Del Papa calculus extraction")
    freezer=freezer.replace("A1+A2","A1 + A2")
    freezer=freezer.replace("B1+B2","B1 + B2")
    freezer=freezer.replace("sub-bag B1+B2 ","")
    freezer=freezer.replace("sub-bag B1 + B2 ","")
    freezer=freezer.replace("sub-bag ","")
    freezer=freezer.replace("pulps","pulp")
    freezer=freezer.replace("roots","root")
    freezer=freezer.replace(" for back-up"," back-up")
    freezer=freezer.replace(" for extraction"," extraction")
    freezer=freezer.replace(" to extract"," extraction")
    freezer=freezer.replace("freezer","Freezer")
    freezer=freezer.replace("Thomas","Tom")
    freezer=freezer.replace("Hannes'","Hannes")
    freezer=freezer.replace("Hanness","Hannes")
    freezer=freezer.replace("Miren drawer","Miren Drawer 2")
    freezer=freezer.replace("blue rack","Blue Rack 1")
    freezer=freezer.replace(", front extraction clean room 159","")
    freezer=freezer.replace("Neme San Rafael","neme san rafael")
    freezer=freezer.replace(", bag mix batch,",", bag Mix batch,")
    freezer=freezer.replace("bag C group sensitive, blue box (back-up)","bag A1 + A2, C group sensitive, blue box, back-up")
    if freezer not in storage:
        print(freezer+" not registered in eLab")
        break
    r=requests.get(url+"/samples/get?sampleID="+idEx,headers=headers2)
    BadRequest(r,200)
    storedIn=r.json()[0]["storageLayerID"]
    needToMove=True
    if storedIn != 0:
        if storageRev[storedIn] != freezer:
            print(name+" already in storage: "+storageRev[storedIn])
            prompt=defaultPrompt
            while prompt not in ["y","n"]:
                prompt=input("Do you want to move it to "+ freezer+"? y/n")
                print(prompt)
            if prompt == "n":
                needToMove=False
    if needToMove:
        IDsto=format(storage[freezer])
        r=requests.post(url+"/samples/moveToLayer/"+IDsto+"?sampleIDs="+idEx,headers=headers1,data={})
        BadRequest(r,204)


print("finished")



finished


## Add description from GeneralSampleComment to Skeletal Element




In [37]:
alreadyUpdated={}
for index,rasID in extractTable['RascovanLabID'].items():
    if format(rasID)=="nan":
        continue
    if rasID in alreadyUpdated.keys():
        continue
    element=extractTable.loc[index,"GeneralSampleComment"]
    r=requests.get(url + "samples/"+registered["Skeleton Element"][rasID], headers = headers2)
    if BadRequest(r,200):
       r.raise_for_status() 
    elementLoaded=r.json()["description"]
    if(format(elementLoaded)!="nan"):
        element=elementLoaded+"| Comment from Drilling Session: "+element
    DR=requests.patch(url + "samples/"+registered["Skeleton Element"][rasID], headers = headers2,data = {"description":element})
    if BadRequest(DR,204):
        DR.raise_for_status()
    alreadyUpdated[rasID]=element
print("finished")
    



finished


In [38]:
alreadyUpdated

{'AR0079.1': 'Mariano del Papa, collected from zip-lock bags',
 'AR0080.1': 'Mariano del Papa, collected from zip-lock bags',
 'AR0081.1': 'Mariano del Papa, collected from zip-lock bags',
 'AR0082.1': 'Mariano del Papa, collected from zip-lock bags',
 'AR0083.1': 'Mariano del Papa, collected from zip-lock bags',
 'AR0084.1': 'Mariano del Papa, collected from zip-lock bags',
 'AR0085.1': 'Mariano del Papa, collected from zip-lock bags',
 'AR0086.1': 'Mariano del Papa, collected from zip-lock bags',
 'AR0087.1': 'Mariano del Papa, collected from zip-lock bags',
 'AR0088.1': 'Mariano del Papa, collected from zip-lock bags',
 'AR0089.1': 'Mariano del Papa, collected from zip-lock bags',
 'AR0090.1': 'Mariano del Papa, collected from zip-lock bags',
 'AR0091.1': 'Mariano del Papa, collected from zip-lock bags',
 'AR0092.1': 'Mariano del Papa, collected from zip-lock bags',
 'AR0093.1': 'Mariano del Papa, collected from zip-lock bags',
 'AR0094.1': 'Mariano del Papa, collected from zip-lock